In [32]:
from coinbase.wallet.client import Client
import yaml
import os
import cbpro
import time
import pandas as pd

In [2]:
with open(os.path.expanduser("~/.ssh/go_trader_coinbase_key.yml"), 'r') as stream:
    try:
        coinbase_tokens = yaml.safe_load(stream)['coinbase']
    except yaml.YAMLError as exc:
        print(exc)

In [3]:
client = Client(api_key=coinbase_tokens['passphrase'], api_secret=coinbase_tokens['api_secret'])

KeyError: 'api_key'

### Test some public/private methods

In [15]:
# authenticate
auth_client = cbpro.AuthenticatedClient('gotrader',coinbase_tokens['api_secret'],coinbase_tokens['passphrase'])

- Get all fills

In [17]:
fills_gen = auth_client.get_fills("BTC-USD")
# Get all fills (will possibly make multiple HTTP requests)
all_fills = list(fills_gen)

In [18]:
all_fills

['message']

In [20]:
class myWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url = "wss://ws-feed.pro.coinbase.com/"
        self.products = ["LTC-USD"]
        self.message_count = 0
        print("Lets count the messages!")
    def on_message(self, msg):
        self.message_count += 1
        if 'price' in msg and 'type' in msg:
            print ("Message type:", msg["type"],
                   "\t@ {:.3f}".format(float(msg["price"])))
    def on_close(self):
        print("-- Goodbye! --")

wsClient = myWebsocketClient()
wsClient.start()
print(wsClient.url, wsClient.products)
while (wsClient.message_count < 500):
    print ("\nmessage_count =", "{} \n".format(wsClient.message_count))
    time.sleep(1)
wsClient.close()

TypeError: __init__() missing 1 required keyword-only argument: 'channels'

In [21]:
import cbpro, time
order_book = cbpro.OrderBook(product_id='BTC-USD')
order_book.start()
time.sleep(10)
order_book.close()

-- Subscribed to OrderBook! --

Error: messages missing (29031085062 - 29031085159). Re-initializing  book at sequence.
Error: messages missing (29031085062 - 29031085160). Re-initializing  book at sequence.

-- OrderBook Socket Closed! --
-- OrderBook Socket Closed! --



In [34]:
# [ time, low, high, open, close, volume ]
# granularity = day
historic_rates = auth_client.get_product_historic_rates(product_id='BTC-USD', granularity=86400)

In [48]:
historic_rates[:10]

[[1630972800, 42830.77, 52944.96, 52698.8, 46950.87, 28706.34074323],
 [1630886400, 51020.08, 52802.03, 51789.17, 52698.81, 9571.75471136],
 [1630800000, 49500, 51907.08, 49944.89, 51789.17, 7743.93472308],
 [1630713600, 49400, 50558.75, 50025, 49942.98, 7595.48816499],
 [1630627200, 48349.66, 51064.44, 49279.38, 50025, 14577.22621879],
 [1630540800, 48620.73, 50392.19, 48834.27, 49279.37, 14120.3826407],
 [1630454400, 46537.62, 49125, 47110.33, 48839.86, 13010.83839478],
 [1630368000, 46708.93, 48259.68, 46996.8, 47112.5, 12919.43970212],
 [1630281600, 46866, 48906.51, 48802.58, 46993.71, 12249.23741495],
 [1630195200, 47801, 49667.1, 48942.89, 48802.58, 6935.09511006]]

- verify this returns daily snapshots... It does

In [45]:
pd.to_datetime(1630972800, unit='s')

Timestamp('2021-09-07 00:00:00')

In [46]:
pd.to_datetime(1630886400, unit='s')

Timestamp('2021-09-06 00:00:00')

In [47]:
pd.to_datetime(1630800000, unit='s')

Timestamp('2021-09-05 00:00:00')